# Model Inference on some CASME2 Clips 

In [1]:
import numpy  as np
import pandas as pd
import csv


# visualization libraries
import matplotlib.pyplot as plt
import plotly.express as px

import cv2
import torch
import torch.nn.functional as F
from ResNet.resnet50 import ResNet, resnet50_config

c:\Users\kings\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load model

In [2]:
# Load the saved state dictionary
state_dict = torch.load(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\models\ResNet_Adamfer_aug.pt")
resnet_model = ResNet(resnet50_config, 3)
# Replace the model's state dictionary with the saved state dictionary
resnet_model.load_state_dict(state_dict)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet_model = resnet_model.to(device)
resnet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Live WebCam Demo ResNet

In [3]:
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub01\EP19_06f.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [4]:
def predict_emotion(output):
    while True:
        # Capture a frame
        ret, frame = cap.read()
        facedet = cv2.CascadeClassifier(r'C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\Msc-Project\Codes\haarcascades\haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facedet.detectMultiScale(gray_frame, 1.1, 4)

        if len(faces) == 0:
            print("No faces detected in the frame")
            continue  # Skip processing this frame if no faces are detected

        for x, y, w, h in faces:
            roi_gray = gray_frame[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            facess = facedet.detectMultiScale(roi_gray)
            if len(facess) == 0:
                print("Face not detected")
            else:
                for (ex, ey, ew, eh) in facess:
                    face_roi = roi_color[ey:ey+eh, ex:ex+ew]  # cropping the face

        rgb_image = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
        image = cv2.resize(rgb_image, (224, 224))
        image = image / 255.0  # normalization
        data = torch.from_numpy(image)
        data = data.type(torch.FloatTensor)
        data = data.to(device)
        reshaped_data = data.permute(2, 0, 1)  # Reshape the tensor to have 3 as the first dimension
        reshaped_data = reshaped_data.unsqueeze(0)  # add a fourth dimension
        outputs = resnet_model(reshaped_data)
        pred = F.softmax(outputs[0], dim=1)
        final_pred = torch.argmax(pred)

        if final_pred == 0:
            emotion = "Negative"
        elif final_pred == 1:
            emotion = "Neutral"
        else:
            emotion = "Positive"

        text_x, text_y = x, y - 10  # Adjust the position above the bounding box
        cv2.putText(frame, emotion, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

        conf, classes = torch.max(pred, 1)
        emotion_id = [0, 1, 2]
        class_dict = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        class_names = [class_dict[label] for label in emotion_id]
        result_data = []
        result_data.append([conf.item(), pred.tolist()[0], class_names[classes.item()]])

        # Write the result to a CSV file
        with open(output+'.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0:
                writer.writerow(['Confidence', 'max_confidence', 'Emotion'])  # Write header only if the file is empty
            writer.writerows(result_data)

        cv2.imshow("Spotting Facial Expression", frame)

        if cv2.waitKey(2) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [5]:
predict_emotion('sub01_EP19_06f')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [7]:
df = pd.read_csv('sub01_EP19_06f.csv')

In [8]:
def plot_predictions(data, clip):
    df = data
    # Filter the data by emotions
    negative_data = df[df['Emotion'] == 'Negative']
    neutral_data = df[df['Emotion'] == 'Neutral']
    positive_data = df[df['Emotion'] == 'Positive']

    # Create a Plotly figure
    fig = px.line(df, x=df.index, y='Confidence', color='Emotion',
                labels={'index': 'Frame', 'Confidence': 'Confidence'},
                title=f'Confidence Scores Over Time for Different Emotions on {clip} clip')

    # Customize the figure appearance (optional)
    fig.update_traces(mode='lines+markers')
    fig.update_layout(legend_title='Emotion')
    fig.update_xaxes(title_text='Frame')
    fig.update_yaxes(title_text='Confidence')

    # # Show the interactive plot
    fig.show()


In [9]:
plot_predictions(df, 'sub01_EP19_06f')

In [10]:
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub26\EP07_37.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [11]:
predict_emotion('sub26_EP07_37')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [12]:
df = pd.read_csv('sub26_EP07_37.csv')

In [13]:
plot_predictions(df, 'sub26_EP07_37')

In [14]:
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub14\EP09_06.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [15]:
predict_emotion('sub14_EP09_06')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [16]:
df = pd.read_csv('sub14_EP09_06.csv')

In [17]:
plot_predictions(df, 'sub14_EP09_06')

In [18]:
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub09\EP15_05.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [19]:
predict_emotion('sub09_EP15_05')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [20]:
df = pd.read_csv('sub09_EP15_05.csv')

In [21]:
plot_predictions(df, 'sub09_EP15_05')

# Live WebCam Demo Inception-V3

In [22]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 3)
state_dict = torch.load(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\models\Inception3_Adamproposed_aug.pt")

model.load_state_dict(state_dict)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

Using cache found in C:\Users\kings/.cache\torch\hub\pytorch_vision_v0.10.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [23]:
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub01\EP19_06f.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [24]:
def predict_emotion(output):
    while True:
        # Capture a frame
        ret, frame = cap.read()
        facedet = cv2.CascadeClassifier(r'C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\Msc-Project\Codes\haarcascades\haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facedet.detectMultiScale(gray_frame, 1.1, 4)

        if len(faces) == 0:
            print("No faces detected in the frame")
            continue  # Skip processing this frame if no faces are detected

        for x, y, w, h in faces:
            roi_gray = gray_frame[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            facess = facedet.detectMultiScale(roi_gray)
            if len(facess) == 0:
                print("Face not detected")
            else:
                for (ex, ey, ew, eh) in facess:
                    face_roi = roi_color[ey:ey+eh, ex:ex+ew]  # cropping the face

        rgb_image = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
        image = cv2.resize(rgb_image, (299, 299))
        image = image / 255.0  # normalization
        data = torch.from_numpy(image)
        data = data.type(torch.FloatTensor)
        data = data.to(device)
        reshaped_data = data.permute(2, 0, 1)  # Reshape the tensor to have 3 as the first dimension
        reshaped_data = reshaped_data.unsqueeze(0)  # add a fourth dimension
        outputs = model(reshaped_data)
        pred = F.softmax(outputs[0], dim=-1)
        final_pred = torch.argmax(pred, 0)

        if final_pred == 0:
            emotion = "Negative"
        elif final_pred == 1:
            emotion = "Neutral"
        else:
            emotion = "Positive"

        text_x, text_y = x, y - 10  # Adjust the position above the bounding box
        cv2.putText(frame, emotion, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

        conf, classes = torch.max(pred, -1)
        emotion_id = [0, 1, 2]
        class_dict = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        class_names = [class_dict[label] for label in emotion_id]
        result_data = []
        result_data.append([conf.item(), pred.tolist(), class_names[classes.item()]])

        # Write the result to a CSV file
        with open(output+'inception.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0:
                writer.writerow(['Confidence', 'max_confidence', 'Emotion'])  # Write header only if the file is empty
            writer.writerows(result_data)

        cv2.imshow("Facial Expression Recognition", frame)

        if cv2.waitKey(2) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [25]:
predict_emotion('sub01_EP19_06f')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [26]:
df = pd.read_csv('sub01_EP19_06finception.csv')
df

,Confidence,max_confidence,Emotion
0,0.567231,"[0.5672310590744019, 0.4019140303134918, 0.030...",Negative
1,0.550232,"[0.5502319931983948, 0.4167941212654114, 0.032...",Negative
2,0.553325,"[0.5533254146575928, 0.41508302092552185, 0.03...",Negative
3,0.564934,"[0.5649341940879822, 0.4047926366329193, 0.030...",Negative
4,0.548058,"[0.5480582118034363, 0.4191238582134247, 0.032...",Negative
...,...,...,...
295,0.557083,"[0.557083010673523, 0.4106135666370392, 0.0323...",Negative
296,0.562786,"[0.5627858638763428, 0.4046153128147125, 0.032...",Negative
297,0.556344,"[0.5563441514968872, 0.409857839345932, 0.0337...",Negative
298,0.577781,"[0.5777808427810669, 0.39224401116371155, 0.02...",Negative


In [28]:
plot_predictions(df, 'sub01_EP19_06finception')

In [31]:
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub26\EP07_37.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [32]:
predict_emotion('sub26_EP07_37')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [33]:
df = pd.read_csv('sub26_EP07_37inception.csv')

In [34]:
plot_predictions(df, 'sub26_EP07_37inception')

In [35]:
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub14\EP09_06.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [36]:
predict_emotion('sub14_EP09_06')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [37]:
df = pd.read_csv('sub14_EP09_06inception.csv')

In [38]:
plot_predictions(df, 'sub14_EP09_06inception')

In [39]:
cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub09\EP15_05.avi")
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [40]:
predict_emotion('sub09_EP15_05')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [41]:
df = pd.read_csv('sub09_EP15_05inception.csv')

In [42]:
plot_predictions(df, 'sub09_EP15_05inception')

: 